### 공동주택관리 정보시스템
* 2018.08, 서울 강남구, 삼성동 소재 모든 아파트,
  아파트 명칭, 법정동 주소,
  아파트 주차대수 ( 지상, 지하 )등을 추출
* 단, 실제 URL에 질의문자열을 포함시켜 정보 추출

### K-atp.go.kr URL 분석
1. http://www.k-apt.go.kr/kaptinfo/openkaptinfo.do
    + 요청정보가 없음
2. getBjList.do : 요청 폼 데이터 포함 ( 지역 )
    + ajax로 작동, 시도선택 -> 시군구 출력 -> 읍면동 출력
    + ?bjd_gbn=SIDO ( 광역시도 출력 )
    + ?bjd_gbn=SGG&bjd_code=11 ( 서울특별시 선택시, 시군구 출력 )
    + ?bjd_gbn=EMD&bjd_code=11680 ( 서울특별시 강남구 선택시 읍면동 출력 )
    + 읍면동 선택시 bjd_code와 search_date 변수를 생성한후 getKaptList.do를 호출
3. getKaptList.do : 요청 폼 데이터 포함 ( 지역내 모든 아파트 )
    + ?bjd_code=11680105&search_date=201807
4. getKaptSetday.do : 요청 폼 데이터 포함 ( 아파트 상세정보 )
    + ?kapt_code=A10026350

In [59]:
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.support.select import Select
from selenium.webdriver.common.by import By

In [60]:
import sys
import time

In [61]:
# 단지검색 키워드
year = '2018'
month = '08'
sido = '서울특별시'
gugn = '강남구'
dong = '삼성동'

In [62]:
base = 'http://www.k-apt.go.kr/'
url1 = base + 'kaptinfo/openkaptinfo.do'
url2 = base + 'cmmn/bjd/getBidList.do'
url3 = base + 'kaptinfo/getKaptList.do'
url4 = base + 'kaptinfo/getKaptSetday.do'
path = '/home/hadoop/chromedriver'

In [63]:
# 셀레니엄 실행
opts = Options()
opts.add_argument('--headless')
chrome = webdriver.Chrome(executable_path=path, options=opts)
chrome.get(url1)
chrome.implicitly_wait(2)

/opt/miniconda3/lib/python3.7/site-packages/ipykernel_launcher.py:4: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  after removing the cwd from sys.path.


In [64]:
yy = Select(chrome.find_element(By.CLASS_NAME, 'combo_YYYY'))
yy.select_by_visible_text(year)
time.sleep(1)

In [65]:
mm = Select(chrome.find_element(By.CLASS_NAME, 'combo_MM'))
mm.select_by_visible_text(month)
time.sleep(1)

In [66]:
sd = Select(chrome.find_element(By.CLASS_NAME, 'combo_SIDO'))
sd.select_by_visible_text(sido)
time.sleep(1)

In [67]:
gg = Select(chrome.find_element(By.CLASS_NAME, 'combo_SGG'))
gg.select_by_visible_text(gugn)
time.sleep(1)

In [68]:
dg = Select(chrome.find_element(By.CLASS_NAME, 'combo_EMD'))
dg.select_by_visible_text(dong)
time.sleep(1)

In [69]:
# 선택한 시구동에 대한 전체코드값 txt_bjdcode 출력
# 단, 이 코드값은 숨겨져 있기 때문에 요소값을 출력할 수 없음
# 따라서, 자바스크립트로 display:none을 display:block으로 바꿔야 함 
bjd = chrome.find_element(By.CLASS_NAME, 'txt_bjdCode')
chrome.execute_script('arguments[0].style.display="block"', bjd)
print(bjd.text)

11680105


In [70]:
# 선택한 시구동에 존재하는 모든 아파트중 
# ' 삼성동센트럴아이파크' 의 코드값 KAPT_CODE 출력
# 단, 이 코드값은 숨겨져 있기 때문에 요소값을 출력할 수 없음
td1 = chrome.find_element(By.XPATH,'//tr[@id="1"]//td[1]')
chrome.execute_script('arguments[0].style.display="block"', td1)
print(td1.text)

A10026350


In [71]:
# 선택한 시구동에 존재하는 모든 아파트중 코드값 KAPT_CODE 출력
cnt = chrome.find_element(By.CSS_SELECTOR, 
                          'table[id="aptInfoList"] tr:last-child').get_attribute('id')
cnt = int(cnt)
for i in range(1, cnt+1):
    code = chrome.find_element(By.XPATH, f'//tr[@id="{i}"]/td[1]')
    chrome.execute_script('arguments[0].style.display="block"', code)
    print(code.text)
    time.sleep(1)

A10026350
A13550701
A13570501
A13509005
A13509007
A13509010
A13509006
A13509101
A13509003
A13509004
A13509001
A13586602
A13509012
A13587603
A13509009


In [72]:
# ' 삼성동센트럴아이파크 ' 의 주차장 정보 출력
kapt_code = td1.text
url4 = url4 + '?kapt_code=' + kapt_code

chrome.get(url4)
chrome.page_source[:500]

'<html><head></head><body>{"kaptSetDayList":[{"ktown_ho_name":"101","ktown_dong_name":"301","ktown_apt_name":"??????????","file_yn":"Y","setday":"2018-11-13","file_name":"301-101.pdf","seq":9601195},{"ktown_ho_name":"103","ktown_dong_name":"301","ktown_apt_name":"??????????","file_yn":"Y","setday":"2018-04-09","file_name":"301-103.pdf","seq":9601196},{"ktown_ho_name":"104","ktown_dong_name":"301","ktown_apt_name":"??????????","file_yn":"Y","setday":"2018-05-10","file_name":"301-104.pdf","seq":960'

In [81]:
# selenium 으로 불러온 소스를 json형식으로 다시 읽어옴
import json
from bs4 import BeautifulSoup
html = BeautifulSoup(chrome.page_source)
apt = json.loads(html.find('body').text)

print(apt.get("kaptInfo").get("KAPTD_PCNT"))
print(apt.get("kaptInfo").get("KAPTD_PCNTU"))

0
743
